# CustomGUI class

Written by Toshiya Sato, Matlantis Corp.(JP) <br>

## CustomGUIについて

.py形式のファイルに格納した関数を呼び出し、GUIで実行するクラス<br>
※ 現在開発中のため、不具合が発生する可能性がございます。あらかじめご承知おきください。

In [1]:
# !pip install -U pfcc-extras  # pfcc-extrasのインストール

## 関数をGUI化するための事前準備

.pyファイルのテンプレート：./code/utils_files/utils_templete.py
1. GUI化する関数とその関数を実行するために必要なimport文を、上記のテンプレートをコピーした.pyファイルに記載
2. 必要に応じて、関数の詳細設定を.pyファイルの末尾に記載（"./code/utils_files/utils_templete.py"参照）

例).<br>
makesurface関数は表面を作成する関数です。

In [4]:
# makesurface関数のimport文と関数
from ase import Atoms
from ase.build import surface

def makesurface(
    atoms: Atoms, miller_indices=(1, 1, 1), layers=4, rep=[4, 4, 1]
) -> Atoms:
    s1 = surface(atoms, miller_indices, layers)
    s1.center(vacuum=10.0, axis=2)
    s1 = s1.repeat(rep)
    s1.set_positions(
        s1.get_positions() - [0, 0, min(s1.get_positions()[:, 2])]
    )
    s1.pbc = True
    return s1

この関数は下のセルのように実行できますが、CustomGUIでは、上記のimport文およびmakesurface関数を.py形式のファイルに格納することで、GUI化することが可能です。<br>
テンプレートとして、"./code/utils_files/utils_templete.py"に格納してありますので、ご確認ください。  

なお、この例で使用する構造ファイルは[CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)の下でMaterials Projectから入手したものです。  
Rh (mp-74, Conventional Cell)

In [5]:
# 通常のmakesurface関数の実行
from ase.io import read
from ase.visualize import view
import ipywidgets as widgets

atoms_before = read("./input/Rh_mp-74_conventional_standard.cif")
atoms_after = makesurface(atoms_before, miller_indices=(1, 1, 1), layers=4, rep=[4, 4, 1])

v1=view(atoms_before,viewer="ngl")
v1.view.add_representation("ball+stick")
v2=view(atoms_after,viewer="ngl")
v2.view.add_representation("ball+stick")

widgets.HBox([v1.view, v2.view])

## CustomGUIの実行

In [2]:
# CustomGUIのimport
import sys
sys.path.append("/home/jovyan/custom_gui/code/")  # customgui.pyファイルが存在するパスに変更してください
from customgui import CustomGUI

%load_ext autoreload
%autoreload 2

In [3]:
# .pyファイルのテンプレートによる動作確認。makesurface関数を実行する場合は、GUI左タブの「Read Atoms」からcifファイルを読み込んでmakesurface関数を選択してください。その後、引数を選択し、Runボタンを押すと実行できます。
v=CustomGUI(utils_path="./code/utils_files/utils_templete.py")
v.display()

In [29]:
# atomsを取り出すには、writeボタンでwriteする以外に以下でも取り出せる
atoms = v.atoms
all_atoms = v.all_atoms

print("atoms    :",atoms)
print("all_atoms:",all_atoms)

atoms    : Atoms(symbols='CH4', pbc=False, calculator=ASECalculator(...))
all_atoms: [Atoms(symbols='CH4', pbc=False, calculator=ASECalculator(...))]


### 使用例1 : モデリング関数
モデリングに使用可能な関数をまとめています
- smiles_to_atoms: SMILESからAtomsに変換
- generate_conformers: 指定したSMILESの分子のコンフォマーを生成 ([pfcc-extrasの機能紹介](https://matlantis.zendesk.com/hc/ja/articles/37243153882521-pfcc-extras%E3%81%AE%E6%A9%9F%E8%83%BD%E7%B4%B9%E4%BB%8B-v0-12-0#h_01H975X5Z6XKM21HBWCPQ5SF6A))
- opt_cell_size: バルクのセルサイズを最適化
- makesurface: バルクから表面を作成
- f_set_constraints: 指定したindexに原子が動かないように固定 (=FixAtomsをかける)
- f_add_adsorbate: slabに対して、吸着分子adsorbateを指定した高さ・位置に吸着させる
- f_liquid_generator: 溶液系を作成 ([pfcc-extrasの機能紹介](https://matlantis.zendesk.com/hc/ja/articles/37243153882521-pfcc-extras%E3%81%AE%E6%A9%9F%E8%83%BD%E7%B4%B9%E4%BB%8B-v0-12-0#h_01H975X5Z6XKM21HBWCPQ5SF6A))
- myopt: 構造最適化の関数
- AddEditor_func: 分子の移動、回転、追加などが可能なモデリングツール ([pfcc-extrasの機能紹介](https://matlantis.zendesk.com/hc/ja/articles/37243153882521-pfcc-extras%E3%81%AE%E6%A9%9F%E8%83%BD%E7%B4%B9%E4%BB%8B-v0-12-0#h_01H975X5Z6XKM21HBWCPQ5SF6A))
- get_energy_plot: Atomsのエネルギーをグラフ化して出力

In [7]:
# 例1).モデリング関数
v=CustomGUI(read("./input/Rh_mp-74_conventional_standard.cif"),  # ここでatomsを指定しなくても"Read Atoms"からreadすることもできます
            utils_path="./code/utils_files/utils1_modeling.py",
            added_import_functions=[("smiles_to_atoms",0),("generate_conformers",0)]  # .pyファイルでimportした関数もドロップダウンで選択可能に
           )
v.display()

### 使用例2 : 物性計算
物性計算に使用可能な関数をまとめています
- neb_calc: 初期構造と終構造を指定してNEB計算を実行。一例として、input/is.jsonとinput/fs.jsonをご用意していますので、ISとFSに指定して挙動をご覧ください。
- nvt_ast6_1: NVT計算を実行 (参照: [Atomistic Simulation Tutorial 6-1](https://docs.matlantis.com/atomistic-simulation-tutorial/ja/6_1_md-nve.html))
- show_result: NEB計算やMD計算の結果のエネルギーダイアグラムを図示

In [ ]:
# !pip install asap3  # MD計算に使用します

In [10]:
# 例2). 物性計算
v=CustomGUI(utils_path="./code/utils_files/utils2_property_calc.py")
v.display()

### 使用例3 : 可視化関数の一元化
可視化のための関数をまとめています。可視化関数の引数指定は現在できません（開発中です）。
- view_func: view_nglによる一般的な可視化方法
- AddEditor_func: 分子の移動、回転、追加などが可能なモデリングツール
- view_charge_scale: Bader電荷の大きさで原子の色が変化するviewer
- lotsatoms_view: 大規模原子(約5000原子まで)の結合を描画可能 ([matlantis-contrib/visualize_big_atoms](https://redirect.matlantis.com/api/docs/ja/matlantis-contrib/visualize_big_atoms/lotsatomsview.html))
- get_energy_plot: Atomsのエネルギーをグラフ化して出力

In [11]:
# 例3). view関数
v=CustomGUI(utils_path="./code/utils_files/utils3_viewer.py")
v.display()

### 使用例4 : NEB計算をGUIで
モデリング、物性計算、可視化を組み合わせることで、例えばバルクのcifファイルと吸着分子のSMILESからNEB計算を実行することができます。  
この例では、分子指定部分は引数の指定が必要ですが、それ以外の引数の値は変更せずにRunボタンを押すことで、NEB計算を実行できます。
- f0_smiles_to_atoms: 吸着分子のSMILESを指定してください。作成した構造はGUI左タブの「Write Atoms」から保存してください。  
  Read Atomsから./input/Rh_mp-74_conventional_standard.cifを読み込んでください。
- f1_1_repeat_bulk:　Rhバルクをスーパーセルに変換します
- f2_makesurface: スーパーセルから表面を作成します
- f3_set_constraints: 指定した高さ以下の原子を固定します  
    ※ viewer下の「Show constraints」チェックボックスを押すことで、固定された原子の可視化が可能です
- f4_add_adsorbate: adsorbateにはf0_smiles_to_atomsで作成した吸着分子を指定してください。吸着分子をスラブに吸着させます。  
    ※ adsorbateドロップダウンのファイルがリロードされていない場合は、右のリロードボタンを押してください
- f5_myopt: 構造最適化します。NEB計算の終構造として「Write Atoms」から保存してください。
- f6_AddEditor: 初期構造を作成します。今回はCO分子をスラブから離します。CO分子のindexである64,65を「Selected atoms」に記入した後に、「Editing > Move」タブでZ+ボタンを数回押して8Å程度離します。Moveの横のスライダーが一回ボタンを押した際に動かす距離なので、2.0Åにすると4回押すことで吸着分子が元の位置から8Å離れます。  
    ※ スラブから離しすぎると周期境界の逆側のRhスラブに吸着する可能性があるのでご注意ください
- f7_myopt: 初期構造を構造最適化します。NEB計算の終構造として「Write Atoms」から保存してください。
- f8_neb_preparation: 初期構造と終構造を一つのAtomsにまとめます。
- f9_neb_calc: NEB計算を実行します。計算結果がviewerに反映され、吸着の様子を確認可能です。
- f10_show_neb_result: NEB計算結果のエネルギーダイアグラムを表示します

In [14]:
# 例4). NEBの実行までを一つのGUIで
v=CustomGUI(utils_path="./code/utils_files/utils4_for_neb.py")
v.display()